In [ ]:
# 

In [1]:
# add signatures

ln <- load("../../exp_res_ks_20191002.RData")

ln

ln2 <- load("signature_list_twas_significant.RData")

head(up_signature_df)

signature_list_twas_significant

# calculating the score
source("ks_connectivity.R")
source("correlation_connectivity.R")

# main function ks score
ks_score <- function(up_signature, down_signature, rank_matrix, compound_name) {
    n = 1
    connectivity_score_vec <- c()
    gene_rank_bing <- rank_matrix
    for (i in 1:ncol(gene_rank_bing)) {
        if (i %% 500 == 0)  { print(paste0("INFO: ", i, " Instances."))}
        pert_vec <- gene_rank_bing[, i]
        up_signature <- up_signature[up_signature %in% pert_vec]   
        down_signature <- down_signature[down_signature %in% pert_vec]
        # get teh rank
        up_v <- match(up_signature, pert_vec)
        down_v <- match(down_signature, pert_vec)
        tmp_s <- .s(up_v, down_v, nrow(gene_rank_bing))
        connectivity_score_vec <- c(connectivity_score_vec, tmp_s)
        
    }
    output_df <- data.frame(compound_name = compound_name, connectivity_score = connectivity_score_vec)
    return(output_df)
    
}




#ks_score_vec <- ks_score(up_signature_df$entrezgene_id, down_signature_df$entrezgene_id, gene_rank_bing, compound_name)

source("../utilities//gene_id.R")
library(annotables)
library(dplyr)

# map the id
id_map <- function(id_vec, from_ref, to_ref, annotable = grch38) {
    id_df <- data.frame(raw_id = id_vec)
    colnames(id_df) <- from_ref
    id_df[, from_ref] <- as.character(id_df[, from_ref])
    by_str <- rlang::set_names(quo_name(from_ref), quo_name(from_ref))
    output_df <- id_df %>%
        left_join(annotable, by = by_str) %>%
        distinct_(from_ref, .keep_all = T) %>%
        select_(to_ref)
    return(output_df[, to_ref])                              
    
} 

extract_signature <- function(df) {
    up_idx <- df$zscore > 0
    down_idx <- !up_idx
    up_df <- df[up_idx, ]
    down_df <- df[down_idx, ]
    up_df$entrez <- id_map(up_df$gene, from_ref = "symbol", to_ref = "entrez")
    head(up_df)
    down_df$entrez <- id_map(down_df$gene, from_ref = "symbol", to_ref = "entrez")
    up_df <- up_df[complete.cases(up_df),]
    down_df <- down_df[complete.cases(down_df),]
    output_list <- list(up_df = up_df, down_df = down_df)
    return(output_list)
}

load("../../signature_list_twas_significant_unique.RData")


merged_up_signature_df <- data.frame(entrez = up_signature_df$entrezgene_id, pvalue_adj = up_signature_df$P_adjusted.x)
merged_down_signature_df <- data.frame(entrez = down_signature_df$entrezgene_id, pvalue_adj = down_signature_df$P_adjusted.x)



merge_signature <- function(merged_up_signature_df, merged_down_signature_df, trait_str_vec) {
    for (trait_str in trait_str_vec) {
        cat(trait_str)
        output_list <- extract_signature(signature_list_twas_significant_unique[[trait_str]])
        tmp_up_df <- output_list$up_df
        tmp_down_df <- output_list$down_df
        tmp_up_df <- tmp_up_df[,c("entrez", "pvalue_adj")]
        tmp_down_df <- tmp_down_df[,c("entrez", "pvalue_adj")]
        merged_up_signature_df <- rbind(merged_up_signature_df, tmp_down_df)
        merged_down_signature_df <- rbind(merged_down_signature_df, tmp_up_df)
    }
    merged_up_signature_df <- merged_up_signature_df %>% arrange(pvalue_adj)
    merged_down_signature_df <- merged_down_signature_df %>% arrange(pvalue_adj)
    output_list <- list(merged_up_signature_df = merged_up_signature_df, merged_down_signature_df = merged_down_signature_df)
    return(output_list)
    
    
}



check_overlap <- function(gsea_df, nash_drugs) {
    lincs_drugs <- tolower(gsea_df$compound_name)

    midx <- na.omit(match(nash_drugs, lincs_drugs))
    overlapped_drugs <- lincs_drugs[midx]
    sum_overlap <- ifelse(lincs_drugs %in% overlapped_drugs, 1, 0)
    #sum_overlap_top5 <- ifelse(lincs_drugs[(5774-289+1):5774] %in% overlapped_drugs, 1, 0)
    sum_overlap_top5 <- ifelse(lincs_drugs[1:289] %in% overlapped_drugs, 1, 0)
    #sum_overlap_top10 <- ifelse(lincs_drugs[(5774-578+1):5774] %in% overlapped_drugs, 1, 0)
    sum_overlap_top10 <- ifelse(lincs_drugs[1:578] %in% overlapped_drugs, 1, 0)
    sum_overlap_top_all <- ifelse(lincs_drugs %in% overlapped_drugs, 1, 0)
    return(c(sum(sum_overlap_top5), sum(sum_overlap_top10)))
}





enrichment_phyper <- function(top_5_count, top_10_count, overlapped_count, total_count ) {
    p_5 <- phyper(top_5_count, overlapped_count, total_count-overlapped_count, round(total_count*0.05), lower.tail = F)
    p_10 <- phyper(top_5_count, overlapped_count, total_count-overlapped_count, round(total_count*0.10), lower.tail = F)
    return(c(p_5, p_10))
} 

overlapped_count <- 30
total_count <- 5774




[1] "up_res_cc"         "up_res_pair_ez"    "down_res_cc"      
 [4] "down_res_pair_ez"  "up_signature_df"   "down_signature_df"
 [7] "gene_rank_bing"    "compound_name"     "ks_score_df_cc"   
[10] "ks_score_df_pair"  "ks_score_df"

gene_id,log2FC,Paired_TTest.x,P_adjusted.x,entrezgene_id,ensembl_gene_id,nash_1,nash_3,nash_5,nash_6,⋯,nonash_11,nonash_13,nonash_14,nonash_15,nonash_16,nonash_19,nonash_20,logFC,Paired_TTest.y,P_adjusted.y
IGFBP2,2.0973080,0.000202895105445194,0.012531722,3485,ENSG00000115457,5.52600986116163,5.60623844775563,5.80123838936584,4.70580320806404,⋯,7.60762971901251,8.21710655454929,6.05645424467979,7.16480158201681,6.88515902007004,5.90900552371629,5.92910472620911,1.45374314504699,0.000202895105445194,0.0125317222900259
CMYA5,1.1595890,0.000351286063494227,0.015516680,202333,ENSG00000164309,5.10754545068052,4.52639783134358,5.06078860316949,4.70682661957452,⋯,5.44503180669064,6.02702125102626,4.41449391819054,5.51407806636564,5.69491675835204,5.4191293368975,5.32986323229981,0.803765816041033,0.000351286063494227,0.0155166800549362
CA14,1.1577054,7.21898619031281e-05,0.008572901,23632,ENSG00000118298,3.92695884103164,4.52667728148627,3.31228804446142,3.7611840986548,⋯,5.80102330076718,5.28823434108233,3.79072847009234,4.65248678502784,4.98535622245193,4.64702533395229,4.38276310543229,0.802460236526014,7.21898619031281e-05,0.0085729014070367
PPARA,0.9425177,3.8419130807681e-06,0.003508627,5465,ENSG00000186951,7.09040716835301,7.94337760121244,7.06933213381294,7.13154404430531,⋯,7.8350016561528,8.12317442534445,7.90121126226443,8.23448266748219,7.70176370316113,7.82085770678669,7.71299357059367,0.653303496229666,3.8419130807681e-06,0.00350862712101147
CYP1A2,0.8565970,0.00102640671975578,0.026312026,1544,ENSG00000140505,9.34437946338902,10.8478859370914,11.040719491202,10.1625169249533,⋯,10.9719082267521,11.5007525062132,11.0083938173085,10.5601374150956,11.1931455636744,10.9987028630248,10.6536886036463,0.593747788836952,0.00102640671975578,0.0263120262966165
CDH23,0.8272671,7.71817164972887e-05,0.008742475,64072,ENSG00000107736,5.23963135189243,6.22435109678499,5.37161154805623,5.10531271851035,⋯,5.99937447328316,5.50561453595297,5.26636501596013,6.00836272358323,6.1225297657928,5.52887612781896,5.65554151422607,0.573417879203735,7.71817164972887e-05,0.00874247473998747


gene,zscore,effect_size,pvalue,pvalue_adj,tissue
C1QTNF4,-8.558122,-34.421578025,1.147205e-17,8.700403e-14,Liver
SH2B1,-8.444623,-0.477798745,3.050285e-17,1.156668e-13,Liver
CENPO,-8.195914,-0.633572522,2.486965e-16,6.287047e-13,Liver
SH2B1,-7.853227,-0.168106459,4.054672e-15,2.315707e-11,Liver_STARNET1
TUFM,-7.742102,-0.492387787,9.778635e-15,2.315707e-11,Liver_STARNET1
EIF3C,7.719595,0.111391350,1.166995e-14,2.315707e-11,Liver_STARNET1
NFATC2IP,7.528287,0.110901132,5.141027e-14,7.651134e-11,Liver_STARNET1
CLN3,7.309086,2.138821340,2.689661e-13,5.099597e-10,Liver
ULK3,-6.953543,-0.118658941,3.562239e-12,4.241201e-09,Liver_STARNET1
PSMC3,6.784423,0.148463886,1.165516e-11,1.156386e-08,Liver_STARNET1



Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [ ]:
trait_vec <- c("Fatty_liver", "FC", "T2D", "CAD")
test_list <- merge_signature(merged_up_signature_df, merged_down_signature_df, trait_vec)

trait_vec <- c("Fatty_liver", "FC", "T2D", "CAD", "BMI_GIANT_2015")
test_list_2 <- merge_signature(merged_up_signature_df, merged_down_signature_df, trait_vec)

test_list_2$merged_up_signature_df

test_list_2$merged_down_signature_df

ks_score_vec <- ks_score(test_list$merged_up_signature_df$entrez, test_list$merged_down_signature_df$entrez, gene_rank_bing, compound_name)

head(ks_score_vec)

ks_score_vec_2 <- ks_score(test_list_2$merged_up_signature_df$entrez, test_list_2$merged_down_signature_df$entrez, gene_rank_bing, compound_name)

head(ks_score_vec_2)

In [ ]:
cor_score_vec <- correlation_score_merged(test_list$merged_up_signature_df$entrez, test_list$merged_down_signature_df$entrez, gene_rank_bing, compound_name)

cor_score_vec_2 <- correlation_score_merged(test_list_2$merged_up_signature_df$entrez, test_list_2$merged_down_signature_df$entrez, gene_rank_bing, compound_name)

drug_str <- "ACARBOSE AMLODIPINE ATORVASTATIN BETAINE CAFFEINE CENICRIVIROC CHOLECALCIFEROL DABIGATRAN DAPAGLIFLOZIN EMRICASAN ERGOCALCIFEROL EXENATIDE EZETIMIBE FENOFIBRATE GLIMEPIRIDE INSULIN GLARGINE INSULIN HUMAN PRADIGASTAT K-877 PX-102 LIRAGLUTIDE QUINIDINE LOSARTAN ROFLUMILAST METFORMIN ROSIGLITAZONE METRELEPTIN ROSUVASTATIN MGL-3196 SELADELPAR MIDAZOLAM SELONSERTIB SEMAGLUTIDE MT-3995 MUROMONAB-CD3 SIMTUZUMAB NAMODENOSON SITAGUPTIN NIVOCASAN SPIRONOLACTONE OBETICHOLIC ACID TELMISARTAN PENTOXIFYLUNE TOFOGLIFLOZIN PERINDOPRIL URSODIOL PIOGLITAZONE"

nash_drugs <- tolower(unlist(stringr::str_split(drug_str, " ")))

length(nash_drugs)



check_overlap(ks_score_vec, nash_drugs)

check_overlap(ks_score_vec_2, nash_drugs)

check_overlap(cor_score_vec, nash_drugs)

check_overlap(cor_score_vec_2, nash_drugs)

head(cor_score_vec)

head(cor_score_vec_2)

In [2]:
trait_vec <- c("Fatty_liver", "FC")
library(dplyr)
calculate_score <- function(trait_vec) {
    test_list <- merge_signature(merged_up_signature_df, merged_down_signature_df, trait_vec)
    ks_score_df <- ks_score(test_list$merged_up_signature_df$entrez, test_list$merged_down_signature_df$entrez, gene_rank_bing, compound_name)
    cor_score_df <- correlation_score_merged(test_list$merged_up_signature_df$entrez, test_list$merged_down_signature_df$entrez, gene_rank_bing, compound_name)
    ks_score_df <- ks_score_df %>% arrange(desc(connectivity_score))
    cor_score_df <- cor_score_df %>% arrange(desc(connectivity_score))
    output_list <- list(ks_score_df = ks_score_df, cor_score_df = cor_score_df)
    return(output_list)
}

In [11]:
test_list <- merge_signature(merged_up_signature_df, merged_down_signature_df, trait_vec)
dim(test_list$merged_up_signature_df)

Fatty_liverFC

[1] 107   2

In [24]:
up_signature_df

gene_id,log2FC,Paired_TTest.x,P_adjusted.x,entrezgene_id,ensembl_gene_id,nash_1,nash_3,nash_5,nash_6,⋯,nonash_11,nonash_13,nonash_14,nonash_15,nonash_16,nonash_19,nonash_20,logFC,Paired_TTest.y,P_adjusted.y
IGFBP2,2.0973080,0.000202895105445194,0.0125317223,3485,ENSG00000115457,5.52600986116163,5.60623844775563,5.80123838936584,4.70580320806404,⋯,7.60762971901251,8.21710655454929,6.05645424467979,7.16480158201681,6.88515902007004,5.90900552371629,5.92910472620911,1.45374314504699,0.000202895105445194,0.0125317222900259
CMYA5,1.1595890,0.000351286063494227,0.0155166801,202333,ENSG00000164309,5.10754545068052,4.52639783134358,5.06078860316949,4.70682661957452,⋯,5.44503180669064,6.02702125102626,4.41449391819054,5.51407806636564,5.69491675835204,5.4191293368975,5.32986323229981,0.803765816041033,0.000351286063494227,0.0155166800549362
CA14,1.1577054,7.21898619031281e-05,0.0085729014,23632,ENSG00000118298,3.92695884103164,4.52667728148627,3.31228804446142,3.7611840986548,⋯,5.80102330076718,5.28823434108233,3.79072847009234,4.65248678502784,4.98535622245193,4.64702533395229,4.38276310543229,0.802460236526014,7.21898619031281e-05,0.0085729014070367
PPARA,0.9425177,3.8419130807681e-06,0.0035086271,5465,ENSG00000186951,7.09040716835301,7.94337760121244,7.06933213381294,7.13154404430531,⋯,7.8350016561528,8.12317442534445,7.90121126226443,8.23448266748219,7.70176370316113,7.82085770678669,7.71299357059367,0.653303496229666,3.8419130807681e-06,0.00350862712101147
CYP1A2,0.8565970,0.00102640671975578,0.0263120263,1544,ENSG00000140505,9.34437946338902,10.8478859370914,11.040719491202,10.1625169249533,⋯,10.9719082267521,11.5007525062132,11.0083938173085,10.5601374150956,11.1931455636744,10.9987028630248,10.6536886036463,0.593747788836952,0.00102640671975578,0.0263120262966165
CDH23,0.8272671,7.71817164972887e-05,0.0087424747,64072,ENSG00000107736,5.23963135189243,6.22435109678499,5.37161154805623,5.10531271851035,⋯,5.99937447328316,5.50561453595297,5.26636501596013,6.00836272358323,6.1225297657928,5.52887612781896,5.65554151422607,0.573417879203735,7.71817164972887e-05,0.00874247473998747
APOF,0.8184490,0.00277134159121504,0.0449692874,319,ENSG00000175336,8.80073897526963,9.83926185982424,8.25067361509282,8.90468709900777,⋯,9.9473883586629,10.1487912002611,9.49206314000751,9.97764734053949,9.40342819983191,9.42750983148548,9.33769179883476,0.567305601377467,0.00277134159121504,0.0449692874301323
SLC6A16,0.8026814,2.77035047400667e-05,0.0063322099,28968,ENSG00000063127,4.59379224325414,5.18989331601924,4.64722634600753,4.91498846698734,⋯,5.57493033755716,5.54510885377195,4.75237283877783,5.51612672590601,5.65726924592276,5.31602869054435,4.98804329358677,0.556376343357094,2.77035047400667e-05,0.00633220985284111
NUDT4,0.7741879,0.000976464422251872,0.0254106823,11163,ENSG00000173598,2.84233661823423,3.76287609373774,3.10951770869996,3.77108868967926,⋯,3.75505793981797,4.14604968095404,3.5148826574574,4.30658257450651,3.92037180724532,3.5865239940187,3.57367849225032,0.536626183025724,0.000976464422251872,0.025410682347185
ATP2B2,0.7691510,2.89503510339492e-05,0.0064094323,491,ENSG00000157087,5.40084506160321,6.36474455125521,5.99513595097404,6.14647918086259,⋯,6.31498866363306,6.69977147142559,6.54043044094481,6.84953727711564,6.8371992805124,6.51104027233598,6.74252643232086,0.533134821717694,2.89503510339492e-05,0.00640943226224342


In [23]:
merged_up_signature_df

entrez,pvalue_adj
3485,0.0125317223
202333,0.0155166801
23632,0.0085729014
5465,0.0035086271
1544,0.0263120263
64072,0.0087424747
319,0.0449692874
28968,0.0063322099
11163,0.0254106823
491,0.0064094323


In [20]:
test_list$merged_up_signature_df$entrez

[1]      3990      1952      1768     23369      5465    137872      6712
  [8]      1071     84166    112817     57669     28968       491      8647
 [15]      7074     23632     64072     54112      2053     10207     23026
 [22]     55219     11059    201164      3485       125      1615     64902
 [29]     10426    131669       801      6660      5236      6482    202333
 [36]    729830     23498       316     22837     55540      9429    118429
 [43]        32     25831      1757        36      9942      1000     23460
 [50]     53637       191      1244     23541     51084     84456      4054
 [57]      4882    116987       462    118980      6767      9110     23368
 [64]    222255      9969     11163      6595      1544      4035     27087
 [71]      9414       323    144193     64324       607      1238       501
 [78]     56922     27332      2887      8309     59348      6580     23254
 [85]      6641     29763      4329     23239     23469     51279     56521
 [92]     80004       126      6594       731      6821      3069    145781
 [99] 100820829     11278     25932       319      5091      2639     84226
[106]     55703     26235

In [22]:
idx <- match(up_signature_df$entrezgene_id, test_list$merged_up_signature_df$entrez)
idx

[1]  25  35  16   5  68  17 102  12  66  13  26   3   7  18  40  15  41  73  14
[20]  45   6 104  43  53  86  28  10  55  36  11  19  91  76  51  30 101  70  46
[39]  21  32 103  71  24  62  47  42  37  31  81  56  34  64  48  75  44  93  39
[58]  52  65  96  58  60  69  22  23  92  49  88 100 107  38  85  29  87  80  95
[77]  20  63  74  54  57  90  79  84  67 106  78  94  33  61  77  59  72  89  97

In [19]:
idx

[1]  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25
[26] 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50
[51] 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75
[76] 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95

In [14]:
up_signature_df$entrezgene_id

[1]   3485 202333  23632   5465   1544  64072    319  28968  11163    491
[11]    125   1768   6712  54112  55540   7074   9429 144193   8647   1757
[21] 137872   2639     32  23541  29763  64902 112817  84456 729830  57669
[31]   2053  56521   1238    191 131669  25932  27087     36  23026   6660
[41]   5091   9414 201164   9110   9942 118429  23498    801   8309   4054
[51]   6482 222255   1000    607  25831    126  22837   1244   9969   6821
[61] 116987 118980   4035  55219  11059  80004  23460  23239  11278  26235
[71]    316   6641  10426   4329   2887    731  10207  23368  64324  51084
[81]   4882  51279  27332  23254   6595  55703  56922   6594   5236   6767
[91]    501    462    323  23469   3069

In [12]:
test_list$merged_up_signature_df

entrez,pvalue_adj
3990,1.622988e-33
1952,8.069106e-06
1768,1.902835e-04
23369,1.698711e-03
5465,3.508627e-03
137872,3.682895e-03
6712,4.516539e-03
1071,4.883212e-03
84166,4.883212e-03
112817,4.901278e-03


In [3]:
score_list <- calculate_score(trait_vec)

Fatty_liver

Warning message:
“`chr_along()` is deprecated as of rlang 0.2.0.
This warning is displayed once per session.”Warning message:
“The `printer` argument is deprecated as of rlang 0.3.0.
This warning is displayed once per session.”Warning message:
“`lang()` is deprecated as of rlang 0.2.0.
Please use `call2()` instead.
This warning is displayed once per session.”Warning message:
“`new_overscope()` is deprecated as of rlang 0.2.0.
Please use `new_data_mask()` instead.
This warning is displayed once per session.”Warning message:
“`overscope_eval_next()` is deprecated as of rlang 0.2.0.
Please use `eval_tidy()` with a data mask instead.
This warning is displayed once per session.”

FC[1] "INFO: 500 Instances."
[1] "INFO: 1000 Instances."
[1] "INFO: 1500 Instances."
[1] "INFO: 2000 Instances."
[1] "INFO: 2500 Instances."
[1] "INFO: 3000 Instances."
[1] "INFO: 3500 Instances."
[1] "INFO: 4000 Instances."
[1] "INFO: 4500 Instances."
[1] "INFO: 5000 Instances."
[1] "INFO: 5500 Instances."
[1] "INFO: 500 Instances."
[1] "INFO: 1000 Instances."
[1] "INFO: 1500 Instances."
[1] "INFO: 2000 Instances."
[1] "INFO: 2500 Instances."
[1] "INFO: 3000 Instances."
[1] "INFO: 3500 Instances."
[1] "INFO: 4000 Instances."
[1] "INFO: 4500 Instances."
[1] "INFO: 5000 Instances."
[1] "INFO: 5500 Instances."


In [4]:
drug_str <- "ACARBOSE AMLODIPINE ATORVASTATIN BETAINE CAFFEINE CENICRIVIROC CHOLECALCIFEROL DABIGATRAN DAPAGLIFLOZIN EMRICASAN ERGOCALCIFEROL EXENATIDE EZETIMIBE FENOFIBRATE GLIMEPIRIDE INSULIN GLARGINE INSULIN HUMAN PRADIGASTAT K-877 PX-102 LIRAGLUTIDE QUINIDINE LOSARTAN ROFLUMILAST METFORMIN ROSIGLITAZONE METRELEPTIN ROSUVASTATIN MGL-3196 SELADELPAR MIDAZOLAM SELONSERTIB SEMAGLUTIDE MT-3995 MUROMONAB-CD3 SIMTUZUMAB NAMODENOSON SITAGUPTIN NIVOCASAN SPIRONOLACTONE OBETICHOLIC ACID TELMISARTAN PENTOXIFYLUNE TOFOGLIFLOZIN PERINDOPRIL URSODIOL PIOGLITAZONE"
nash_drugs <- tolower(unlist(stringr::str_split(drug_str, " ")))
save(nash_drugs, file = "../data/nash_drugs.RData")

In [9]:
check_overlap

function(gsea_df, nash_drugs) {
    lincs_drugs <- tolower(gsea_df$compound_name)

    midx <- na.omit(match(nash_drugs, lincs_drugs))
    overlapped_drugs <- lincs_drugs[midx]
    sum_overlap <- ifelse(lincs_drugs %in% overlapped_drugs, 1, 0)
    #sum_overlap_top5 <- ifelse(lincs_drugs[(5774-289+1):5774] %in% overlapped_drugs, 1, 0)
    sum_overlap_top5 <- ifelse(lincs_drugs[1:289] %in% overlapped_drugs, 1, 0)
    #sum_overlap_top10 <- ifelse(lincs_drugs[(5774-578+1):5774] %in% overlapped_drugs, 1, 0)
    sum_overlap_top10 <- ifelse(lincs_drugs[1:578] %in% overlapped_drugs, 1, 0)
    sum_overlap_top_all <- ifelse(lincs_drugs %in% overlapped_drugs, 1, 0)
    return(c(sum(sum_overlap_top5), sum(sum_overlap_top10)))
}
<bytecode: 0x1ab271a8>

In [5]:
check_overlap(score_list$ks_score_df, nash_drugs)

[1] 2 2

In [9]:
score_list$ks_score_df[score_list$ks_score_df$compound_name == "atorvastatin",]

,compound_name,connectivity_score
1924,atorvastatin,0
3836,atorvastatin,0
3837,atorvastatin,0
3983,atorvastatin,0
3984,atorvastatin,0


In [6]:
check_overlap(score_list$cor_score_df, nash_drugs)

[1] 6 9

In [ ]:
trait_vec <- c("Fatty_liver", "FC", "T2D")
score_list_2 <- calculate_score(trait_vec)
check_overlap(score_list_2$ks_score_df, nash_drugs)

In [10]:
output_list

ERROR: Error in eval(expr, envir, enclos): object 'output_list' not found


In [ ]:
rank_df <- function()

In [ ]:

enrichment_phyper(5, 6, overlapped_count, total_count)

